<a href="https://colab.research.google.com/github/physicaone/loss_IG/blob/master/%5B211029%5DExact_training6_sym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import torch
import torchvision.datasets
import torchvision.models
import torchvision.transforms
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import torch.utils.data
import torch.nn as nn
from tqdm import tqdm, tnrange
import warnings
warnings.filterwarnings("ignore")
import copy

import pickle as pkl
import pandas as pd
from scipy.stats import entropy
CUDA = torch.cuda.is_available()
CUDA_DEVICE = 0
if CUDA:
    device='cuda:1'
else:
    device='cpu'
torch.cuda.is_available()

CUDA = torch.cuda.is_available()
CUDA_DEVICE = 0
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base='drive/MyDrive'
except:
    if torch.cuda.device_count()>1:
        base='.'
    else:
        base='Google Drive'

if CUDA:
    device='cuda:1'
else:
    device='cpu'
torch.cuda.is_available()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


False

In [13]:
def decimal_to_binary(integer, n_hid):
    string=bin(int(integer))[2:]
    list0=[float(d) for d in string]
    while len(list0)<n_hid:
        list0=[0.]+list0
    return torch.tensor([list0])

def Ising_energy(v_list):
    L = 3
    E_list=[]
    for n in range(len(v_list)):
        v=v_list[n]
        E = 0
        for i in range(L):
            for j in range(L):
                s = v[i,j]
                neigh = v[(i+1)%L, j] + v[i,(j+1)%L] + v[(i-1)%L,j] + v[i,(j-1)%L] 
                E += -neigh * s
        E_list.append(E/2)
    return np.array(E_list)
    
class RBM(nn.Module):

    def __init__(self, n_vis, n_hid, k):
        """Create a RBM."""
        super(RBM, self).__init__()
        
        self.Weight = nn.Parameter(std*torch.randn(n_hid, n_vis).to(device))
        self.k = k


    def v2h(self, v):
        return torch.tanh(F.linear(v, self.Weight))

    def h2v(self, h):
        return torch.tanh(F.linear(h, self.Weight.t()))
    
    def Fv(self, v):
        h_term = torch.sum(torch.log(torch.exp(-torch.matmul(v, self.Weight.t()))+torch.exp(torch.matmul(v, self.Weight.t()))), dim=1)
        return -h_term 

    def energy(self, v, h):
        v=v.bernoulli()
        h=h.bernoulli()
        return -torch.matmul(torch.matmul(v, self.Weight.t()),h.t())


    def forward(self, v):

        return v
        
from torch.utils.data import Dataset

class CustomDataset(Dataset): 
    def __init__(self, dataset):
        data_x = dataset
        self.x_data = data_x
#         self.y_data = data_y

    # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.x_data)
    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
#         y = torch.FloatTensor([self.y_data[idx]])
        return x

def data_to_loader(fullconfigs):
    fulldata=CustomDataset(fullconfigs)
    full_dataset = fulldata
    full_loader = torch.utils.data.DataLoader(full_dataset, batch_size)
    return full_loader

def train_and_get_data(n_hid, model, lr):

    rbm=RBM(n_vis, n_hid, k)

    train_loss_list=[]
#     train_op = optim.Adam(rbm.parameters(), lr)
    train_op = optim.SGD(rbm.parameters(), lr, momentum=0.9)
    rbm.train()
    train_loss_list=[]
    model_list=[]
    for epoch in tnrange(n_epochs):
        Fv=torch.dot(rbm.Fv(v_list_ising2).double(), Pv)
        FE=-torch.log(torch.sum(torch.exp(-rbm.Fv(v_list_ising2))))

        train_loss = Fv-FE
        train_op.zero_grad()
        train_loss.backward()
        train_op.step()
        GE=Fv-FE-S

        if epoch in epoch_to_save:
            train_loss_list.append(float((train_loss-S).detach().cpu().numpy()))
            model_list.append(copy.deepcopy(rbm.cpu().state_dict()))
            print('epoch={epoch}, GE={GE}'.format(epoch=epoch, GE=GE))

    return model_list, train_loss_list


In [14]:
# Hyper parameter들을 설정
n_vis=9
k=1
n_epochs=100000
lr=0.1
std=0.5
epoch_to_save=[2**i for i in range(20)]
n_epochs=epoch_to_save[-1]+1


In [ ]:

torch.set_printoptions(precision=10)
for n_hid in [12]:
    v_list_ising=[]
    v_list_ising2=[]
    for s in range(2**n_vis):
        v=decimal_to_binary(s, n_vis)[0]
        v_list_ising.append(np.reshape(v,(3,3))*2-1)
        v_list_ising2.append(v)
    v_list_ising2=torch.stack(v_list_ising2).to(device)*2-1

    for T in [16]:
        bf_list=np.exp(-Ising_energy(v_list_ising)/T)
        S=entropy(bf_list)
        Pv=torch.tensor(bf_list/sum(bf_list)).to(device).double()

        model_dict={}
        error_dict={}
        for m in range(10):
            model0, loss=train_and_get_data(n_hid, 0, lr=0.1)
            model_dict[str(m)]=model0
            error_dict[str(m)]=loss
        with open('{base}/loss_IG/3*3/Exact_state_dict/model_n_hid={n_hid}_T={T}_sym.pkl'.format(base=base, n_hid=n_hid, T=T), 'wb') as f:
            pkl.dump(model_dict, f)
        with open('{base}/loss_IG/3*3/Exact_error/error_n_hid={n_hid}_T={T}_sym.pkl'.format(base=base, n_hid=n_hid, T=T), 'wb') as f:
            pkl.dump(error_dict, f)
